##Inisialisasi

In [1]:
!pip install --upgrade tensorflow 
!pip install tensorflow_decision_forests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 511.7 MB 6.3 kB/s 
     |████████████████████████████████| 438 kB 54.2 MB/s 
     |████████████████████████████████| 1.6 MB 53.6 MB/s 
     |████████████████████████████████| 5.8 MB 43.4 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.

In [2]:
#Impor Package dan Libraries yang dibutuhkan
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import preprocessing
import tensorflow_decision_forests as tfdf

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Impor dataset
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Fintech Demo Day/train.csv', sep=';')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Fintech Demo Day/test.csv', sep=';')

In [4]:
df_train.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [5]:
df_train['age_bin'] = pd.cut(df_train['age'], [0, 20, 30, 40, 50, 60, 100], 
                              labels=['0-20','20-30','30-40','40-50','50-60','60+'])
df_train = df_train.drop('age',axis = 1)

df_test['age_bin'] = pd.cut(df_test['age'], [0, 20, 30, 40, 50, 60, 100], 
                              labels=['0-20','20-30','30-40','40-50','50-60','60+'])
df_test = df_test.drop('age',axis = 1)

In [6]:
df_train = pd.get_dummies(df_train, columns=['job','marital','education','default','housing','loan','contact','month','poutcome','age_bin'], drop_first=False, dtype=np.int64)
df_test = pd.get_dummies(df_test, columns=['job','marital','education','default','housing','loan','contact','month','poutcome','age_bin'], drop_first=False, dtype=np.int64)

In [7]:
df_train['y'].replace(['yes','no'],[1,0], inplace=True)
df_test['y'].replace(['yes','no'],[1,0], inplace=True)

In [8]:
df_train_yes = df_train[df_train['y'] == 1]
df_train = pd.concat([df_train, df_train_yes, df_train_yes])

##Model Random Forest

In [9]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(df_train, label="y")
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(df_test, label="y")

model = tfdf.keras.RandomForestModel()
model.fit(train_ds)

/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core.py:2574: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)


Use /tmp/tmp2gd5nilu as temporary training directory
Reading training dataset...
Training dataset read in 0:00:12.469109. Found 55789 examples.
Training model...
Model trained in 0:01:05.871246
Compiling model...
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model compiled.


In [10]:
model.summary()

Model: "random_forest_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (56):
	age_bin_0-20
	age_bin_20-30
	age_bin_30-40
	age_bin_40-50
	age_bin_50-60
	age_bin_60+
	balance
	campaign
	contact_cellular
	contact_telephone
	contact_unknown
	day
	default_no
	default_yes
	duration
	education_primary
	education_secondary
	education_tertiary
	education_unknown
	housing_no
	housing_yes
	job_admin.
	job_blue-collar
	job_entrepreneur
	job_housemaid
	job_management
	job_retired
	job_self-employed
	job_services
	job_student
	job_technician
	job_unemployed
	job_unknown
	loan_no
	loan_yes
	marital_divorced
	marital_married
	marital_single
	month_apr
	month_aug
	month_dec
	month_feb
	month_jan
	month_jul
	month_ju

In [11]:
#Menentukan confusion matrix dan classification report untuk data prediksi
y_test = df_test['y']

predictions = (model.predict(test_ds) > 0.5).astype("int32")
confusion_matrix(y_test, predictions, labels=[0, 1])

5/5 [==============================] - 0s 63ms/step


array([[3755,  245],
       [  63,  458]])

In [12]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.98      0.94      0.96      4000
           1       0.65      0.88      0.75       521

    accuracy                           0.93      4521
   macro avg       0.82      0.91      0.85      4521
weighted avg       0.95      0.93      0.94      4521

